# CAPSTONE PROJECT

#### *This notebook will mainly be used for the capstone of the Coursera IBM course.*

In [44]:
import pandas as pd
import numpy as np
import urllib.request
import json
import requests

In [45]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Part 1:  Scraping data into a dataframe and cleanup

##### scrape the data on the wikipedia page and read into a pandas dataframe
##### Ignore values not assigned a borough

In [46]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df.columns = ['PostalCode', 'Borough', 'Neighbourhood']

# Ignore values with no borough assigned

df = df[df.Borough != 'Not assigned']
df.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


##### Combine values with same postal code 

In [47]:
df = df.groupby(['PostalCode','Borough'], sort = False).agg(lambda x: ','.join(x))
df.reset_index(level=['PostalCode', 'Borough'], inplace=True)
df.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Not assigned


##### Replace Neighbourhood 'Not assigned' values with Borough value

In [48]:
df.loc[df['Neighbourhood'] == ('Not assigned'), 'Neighbourhood'] = df['Borough']
df.head()

#df.loc[df['Neighbourhood']=='Not assigned']


,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [49]:
df.shape[0]

103

## Part 2: Geocoder API function and DataFrame

In [50]:
csv_url = 'http://cocl.us/Geospatial_data'

In [51]:
df_csv = pd.read_csv(csv_url)

In [52]:
frames = [df, df_csv]

In [94]:
result = pd.concat(frames, axis=1, sort = False)
result

,PostalCode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M1B,43.806686,-79.194353
1,M4A,North York,Victoria Village,M1C,43.784535,-79.160497
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",M1E,43.763573,-79.188711
3,M6A,North York,"Lawrence Heights,Lawrence Manor",M1G,43.770992,-79.216917
4,M7A,Queen's Park,Queen's Park,M1H,43.773136,-79.239476
5,M9A,Etobicoke,Islington Avenue,M1J,43.744734,-79.239476
6,M1B,Scarborough,"Rouge,Malvern",M1K,43.727929,-79.262029
7,M3B,North York,Don Mills North,M1L,43.711112,-79.284577
8,M4B,East York,"Woodbine Gardens,Parkview Hill",M1M,43.716316,-79.239476
9,M5B,Downtown Toronto,"Ryerson,Garden District",M1N,43.692657,-79.264848


this is the new dataframe that has latitude and longitude values ie coordinates

## Part 3: Analysis

In [56]:
# Import required libraries

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


##### Find the coordinates of the city of Toronto

In [114]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


##### Create a map in which neighbourhoods are displayed

In [115]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(result['Latitude'], result['Longitude'], result['Borough'], result['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [116]:
CLIENT_ID = '5NSSQ4P0532GAPJEXRKQTSPOA1B5RJ4XXYNAI5MN533L53FH' # your Foursquare ID
CLIENT_SECRET = 'NMK1K5RMTYEU2CCDQXWS3PUC51AKUOBPYT2URLP1ZNL2SZAH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5NSSQ4P0532GAPJEXRKQTSPOA1B5RJ4XXYNAI5MN533L53FH
CLIENT_SECRET:NMK1K5RMTYEU2CCDQXWS3PUC51AKUOBPYT2URLP1ZNL2SZAH


##### Pick a borough and cluster it

In [117]:
toronto_data = result[result['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",M1E,43.763573,-79.188711
1,M5B,Downtown Toronto,"Ryerson,Garden District",M1N,43.692657,-79.264848
2,M5C,Downtown Toronto,St. James Town,M1W,43.799525,-79.318389
3,M5E,Downtown Toronto,Berczy Park,M2L,43.757490,-79.374714
4,M5G,Downtown Toronto,Central Bay Street,M2R,43.782736,-79.442259


Downtown Toronto picked as Borough to cluster

In [118]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name =toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Harbourfront,Regent Park are 43.7635726, -79.1887115.


define a function to get nearby venues

In [119]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        LIMIT = 300    
        # create the API request URL
        turl = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
            lat, 
            lng, 
            radius,
            LIMIT)
            
        # make the GET request
        results = requests.get(turl).json()["response"]["groups"][0]["items"]
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [120]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Christie
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown,St. James Town
First Canadian Place,Underground city
Church and Wellesley


In [121]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Airport,Athletics & Sports,Auto Workshop,Bakery,Bar,Baseball Field,Beer Store,Bookstore,Breakfast Spot,...,Steakhouse,Summer Camp,Supplement Shop,Sushi Restaurant,Tanning Salon,Tea Room,Trail,Vegetarian / Vegan Restaurant,Video Store,Wings Joint
0,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [125]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Airport,Athletics & Sports,Auto Workshop,Bakery,Bar,Baseball Field,Beer Store,Bookstore,Breakfast Spot,...,Steakhouse,Summer Camp,Supplement Shop,Sushi Restaurant,Tanning Salon,Tea Room,Trail,Vegetarian / Vegan Restaurant,Video Store,Wings Joint
0,"Adelaide,King,Richmond",0.333333,0.0,0.0000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1,Berczy Park,0.000000,0.0,0.0000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.0,0.0625,0.000000,0.000000,0.00,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,"Cabbagetown,St. James Town",0.000000,0.0,0.0000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,Central Bay Street,0.000000,0.0,0.0000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
5,"Chinatown,Grange Park,Kensington Market",0.000000,0.0,0.0000,0.000000,0.026316,0.00,0.0,0.026316,0.000000,...,0.000000,0.000000,0.026316,0.052632,0.000000,0.026316,0.000000,0.026316,0.0,0.000000
6,Christie,0.000000,0.0,0.0000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
7,Church and Wellesley,0.000000,0.0,0.0000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
8,"Commerce Court,Victoria Hotel",0.000000,0.0,0.0000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,...,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.0,0.000000
9,"Design Exchange,Toronto Dominion Centre",0.000000,0.0,0.0000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,...,0.058824,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


Top five venues

In [126]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
          venue  freq
0       Airport  0.33
1          Park  0.33
2      Bus Stop  0.33
3  Liquor Store  0.00
4     Pet Store  0.00


----Berczy Park----
          venue  freq
0     Cafeteria   1.0
1       Airport   0.0
2  Liquor Store   0.0
3     Pet Store   0.0
4          Park   0.0


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
           venue  freq
0            Spa  0.06
1  Garden Center  0.06
2     Smoke Shop  0.06
3     Restaurant  0.06
4     Comic Shop  0.06


----Cabbagetown,St. James Town----
                 venue  freq
0         Home Service   0.5
1  Empanada Restaurant   0.5
2              Airport   0.0
3         Liquor Store   0.0
4            Pet Store   0.0


----Central Bay Street----
            venue  freq
0   Grocery Store   0.2
1     Pizza Place   0.2
2     Coffee Shop   0.2
3  Discount Store   0.2
4        Pharmacy   0.2


----Chinatown,Grange Park,Kensington Market----


Define most common venues

In [127]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [128]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Airport,Bus Stop,Park,Garden,French Restaurant,Convenience Store,Cosmetics Shop,Curling Ice,Dance Studio,Dessert Shop
1,Berczy Park,Cafeteria,Wings Joint,Empanada Restaurant,Convenience Store,Cosmetics Shop,Curling Ice,Dance Studio,Dessert Shop,Diner,Discount Store
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Park,Smoke Shop,Garden Center,Fast Food Restaurant,Farmers Market,Gym / Fitness Center,Light Rail Station,Comic Shop,Pizza Place,Restaurant
3,"Cabbagetown,St. James Town",Home Service,Empanada Restaurant,Wings Joint,Falafel Restaurant,Convenience Store,Cosmetics Shop,Curling Ice,Dance Studio,Dessert Shop,Diner
4,Central Bay Street,Pharmacy,Pizza Place,Grocery Store,Coffee Shop,Discount Store,Electronics Store,Convenience Store,Cosmetics Shop,Curling Ice,Dance Studio


#### Create clusters

In [129]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 0, 2, 0, 0, 0, 0, 4, 0], dtype=int32)

In [130]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighbourhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",M1E,43.763573,-79.188711,0,Electronics Store,Intersection,Mexican Restaurant,Pizza Place,Medical Center,Rental Car Location,Breakfast Spot,Cosmetics Shop,Curling Ice,Dance Studio
1,M5B,Downtown Toronto,"Ryerson,Garden District",M1N,43.692657,-79.264848,0,College Stadium,Skating Rink,Café,General Entertainment,French Restaurant,Food & Drink Shop,Convenience Store,Cosmetics Shop,Curling Ice,Garden Center
2,M5C,Downtown Toronto,St. James Town,M1W,43.799525,-79.318389,0,Chinese Restaurant,Fast Food Restaurant,Coffee Shop,Bubble Tea Shop,Cosmetics Shop,Pharmacy,Pizza Place,Sandwich Place,Indian Restaurant,Nail Salon
3,M5E,Downtown Toronto,Berczy Park,M2L,43.757490,-79.374714,3,Cafeteria,Wings Joint,Empanada Restaurant,Convenience Store,Cosmetics Shop,Curling Ice,Dance Studio,Dessert Shop,Diner,Discount Store
4,M5G,Downtown Toronto,Central Bay Street,M2R,43.782736,-79.442259,0,Pharmacy,Pizza Place,Grocery Store,Coffee Shop,Discount Store,Electronics Store,Convenience Store,Cosmetics Shop,Curling Ice,Dance Studio


### Map with clusters

In [131]:

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

##### Cluster 1

In [132]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,-79.188711,0,Electronics Store,Intersection,Mexican Restaurant,Pizza Place,Medical Center,Rental Car Location,Breakfast Spot,Cosmetics Shop,Curling Ice,Dance Studio
1,Downtown Toronto,-79.264848,0,College Stadium,Skating Rink,Café,General Entertainment,French Restaurant,Food & Drink Shop,Convenience Store,Cosmetics Shop,Curling Ice,Garden Center
2,Downtown Toronto,-79.318389,0,Chinese Restaurant,Fast Food Restaurant,Coffee Shop,Bubble Tea Shop,Cosmetics Shop,Pharmacy,Pizza Place,Sandwich Place,Indian Restaurant,Nail Salon
4,Downtown Toronto,-79.442259,0,Pharmacy,Pizza Place,Grocery Store,Coffee Shop,Discount Store,Electronics Store,Convenience Store,Cosmetics Shop,Curling Ice,Dance Studio
5,Downtown Toronto,-79.329656,0,Park,Food & Drink Shop,Fast Food Restaurant,Wings Joint,Electronics Store,Convenience Store,Cosmetics Shop,Curling Ice,Dance Studio,Dessert Shop
6,Downtown Toronto,-79.464763,0,Airport,Bus Stop,Park,Garden,French Restaurant,Convenience Store,Cosmetics Shop,Curling Ice,Dance Studio,Dessert Shop
7,Downtown Toronto,-79.318389,0,Pharmacy,Beer Store,Cosmetics Shop,Curling Ice,Dance Studio,Bus Stop,Skating Rink,Park,Athletics & Sports,Video Store
8,Downtown Toronto,-79.315572,0,Park,Brewery,Movie Theater,Italian Restaurant,Pet Store,Pizza Place,Pub,Ice Cream Shop,Sandwich Place,Burger Joint
10,Downtown Toronto,-79.476013,0,Convenience Store,Check Cashing Service,Restaurant,Sandwich Place,Wings Joint,Comic Shop,Cosmetics Shop,Curling Ice,Dance Studio,Dessert Shop
11,Downtown Toronto,-79.484450,0,Café,Coffee Shop,Pizza Place,Sushi Restaurant,Italian Restaurant,Food,Food & Drink Shop,French Restaurant,Falafel Restaurant,Gastropub


##### Cluster 2

In [133]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Downtown Toronto,-79.498509,1,Pool,Baseball Field,Locksmith,Business Service,Wings Joint,Electronics Store,Cosmetics Shop,Curling Ice,Dance Studio,Dessert Shop
16,Downtown Toronto,-79.532242,1,Construction & Landscaping,Baseball Field,Falafel Restaurant,Convenience Store,Cosmetics Shop,Curling Ice,Dance Studio,Dessert Shop,Diner,Discount Store


##### Cluster 3

In [134]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Downtown Toronto,-79.565963,2,Home Service,Empanada Restaurant,Wings Joint,Falafel Restaurant,Convenience Store,Cosmetics Shop,Curling Ice,Dance Studio,Dessert Shop,Diner


##### Cluster 4

In [135]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Downtown Toronto,-79.374714,3,Cafeteria,Wings Joint,Empanada Restaurant,Convenience Store,Cosmetics Shop,Curling Ice,Dance Studio,Dessert Shop,Diner,Discount Store


##### Final Cluster 5

In [139]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Downtown Toronto,-79.38316,4,Trail,Playground,Summer Camp,Wings Joint,Electronics Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice,Dance Studio
